In [42]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import sys,json
import os
import sqlalchemy as sa
sys.path.append("../")
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'qaphelaWeb.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from api import models
from django.db.models import Count, Avg, Q, Prefetch, F, Subquery

from IPython.display import clear_output

# API via REST

In [1]:
import requests

In [2]:
pickup_address = {
    "address": "626 Kamdebo Rd, Florauna, Pretoria, 0182, South Africa",
    "city": "Pretoria",
    "country": "South Africa",
    "lattitude": "-25.688660000000002",
    "longitude": "28.164092",
    "postal_code": "182.0",
    "province": "Gauteng",
    "street_name": "Kamdebo Road",
    "street_number": "626.0",
    "suburb": "Florauna"}

In [8]:
contact_details = {
    "primary_contact_number": "0817189287",
    "secondary_contact_number": "0798595080",
    "next_of_kin_firstname": "Mabu",
    "next_of_kin_lastname": "Manaileng",
    "next_of_kin_primary_contact_number": "0798595080"}

In [16]:
from datetime import datetime
import json

In [23]:
headers =  {"Content-Type": "application/json"}
headers

{'Content-Type': 'application/json'}

In [50]:
%%time
payload = {
    "pickup_address":pickup_address,
    "contact_details":contact_details,
    "incident_type": "NOW",
    "pickup_time": str(datetime.now()),
    "victim_full_names": "Mabu Manaileng"}
payload = json.dumps(payload)

req = requests.post(
    "http://localhost:8001/api/fetchme/", data=payload, headers=headers)
req.json()

CPU times: user 2.8 ms, sys: 1.36 ms, total: 4.16 ms
Wall time: 20.4 ms


{'id': 21,
 'incident_type': 'NOW',
 'incident_status': 'WAITING',
 'pickup_address': {'address': '626 Kamdebo Rd, Florauna, Pretoria, 0182, South Africa',
  'city': 'Pretoria',
  'country': 'South Africa',
  'lattitude': '-25.688660000000002',
  'longitude': '28.164092',
  'postal_code': '182.0',
  'province': 'Gauteng',
  'street_name': 'Kamdebo Road',
  'street_number': '626.0',
  'suburb': 'Florauna'},
 'pickup_time': '2020-10-19 18:57:06.777933',
 'victim_full_names': 'Mabu Manaileng',
 'contact_details': {'id': 1,
  'primary_contact_number': '0817189287',
  'secondary_contact_number': '0798595080',
  'next_of_kin_firstname': 'Mabu',
  'next_of_kin_lastname': 'Manaileng',
  'next_of_kin_primary_contact_number': '0798595080',
  'next_of_kin_secondary_contact_number': None},
 'responders': []}

In [49]:
models.FetchMeIncident.objects.all().delete()

(5, {'api.FetchMeIncident': 5})

In [ ]:
models.Fetch

# Using Alchemy

0

In [11]:
# scr_db = {
#     'NAME': 'deephealthdb',
#     'USER': 'deephealth',
#     'PASSWORD': 'd33ph3a1th',
#     # 'HOST': '35.228.3.60',
#     'HOST': 'localhost',
#     'PORT': '5432'
# }

In [12]:
# db = scr_db
# engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
#     user = db['USER'],
#     password = db['PASSWORD'],
#     host = db['HOST'],
#     port = db['PORT'],
#     database = db['NAME'],
# )

# engine = sa.create_engine(engine_string)
# engine.connect()

# Load Addresses

In [10]:
# address = pd.read_sql_table('api_address', engine)
# address.head()

In [5]:
address = address.sample(50)
address.shape

(50, 12)

In [9]:
# cols = list(address.columns)
# cols.remove("id")
# cols.remove("graph_id")
# cols

In [8]:
# for row in address.values:
#     row = list(row)
#     row = {col:val for col, val in zip(cols, row[1:-1])}
#     addr = models.Address.objects.create(
#         address = row['address'],
#         city = row['city'],
#         country = row['country'],
#         lattitude = row['lattitude'],
#         longitude = row['longitude'],
#         postal_code = row['postal_code'],
#         province = row['province'],
#         street_name = row['street_name'],
#         street_number = row['street_number'],
#         suburb = row['suburb'])

# Add DB Objects

In [16]:
from django.contrib.auth.hashers import make_password

In [17]:
user = django.contrib.auth.models.User.objects.create(
    username="0817189287",
    first_name="Mabu",
    last_name="Manaileng",
    is_active=True,
    email="ManailengMJ@gmail.com",
    password=make_password("11235")
)
# user = django.contrib.auth.models.User.objects.get(id=1)
user

<User: 0817189287>

In [4]:
import numpy as np

In [5]:
addresses = models.Address.objects.all()
address = addresses[np.random.randint(len(addresses))]
address

<Address: Address object (46)>

In [20]:
contact_numbers = models.ContactDetails.objects.get_or_create(
    primary_contact_number = '0817189287',
    secondary_contact_number = '0798595080',
    next_of_kin_firstname = 'Mabu',
    next_of_kin_lastname = 'Manaileng',
    next_of_kin_primary_contact_number = '0798595080'
)
social_profile = models.SocialProfiles.objects.get_or_create(
    name="Twitter",
    profile_page="https://twitter.com/thisMabu"
)

quser = models.QaphelaUser.objects.get_or_create(
    user = user,
    user_type = "STANDARD",
    is_dispatch_user = True,
    social_profiles = social_profile[0],
    contact_details = contact_numbers[0],
    address = address)[0]

# quser = models.QaphelaUser.objects.get(user_id=1)

quser

<QaphelaUser: QaphelaUser object (1)>

In [21]:
contact_numbers = models.ContactDetails.objects.get_or_create(
    primary_contact_number = '0727308412',
    # secondary_contact_number = '0798595080',
    # next_of_kin_firstname = 'Mabu',
    # next_of_kin_lasttname = 'Manaileng',
    # next_of_kin_primary_contact_number = '0817189287'
)[0]

abuser = models.Abuser.objects.get_or_create(
    full_names = "Bonginkosi Lukhele",
    # social_profiles = social_profile,
    contact_details = contact_numbers,
    workplace_details = "Head of Mobile Development at Datawizzards as of October 2020")[0]

case = models.Case.objects.get_or_create(
    title = "Assult Charge",
    report_type = "REPORT",
    occasion = "HOME",
    preview = "I dated him for 3 years, and he got addicted to alcohol making me his punch box for all 3 years until I left the relationship.",
    when = "2019-01-01",
    reporter = quser,
    frequency = 1,
    perpetrators = abuser,
    case_status = "VERIFIED",
    settlement_type = "IN COURT",
    police_case_number = "2019/05/123")[0]
case


<Case: Case object (1)>

In [22]:
contact_numbers = models.ContactDetails.objects.get_or_create(
    primary_contact_number = '0716634002',
    # secondary_contact_number = '0798595080',
    # next_of_kin_firstname = 'Mabu',
    # next_of_kin_lasttname = 'Manaileng',
    # next_of_kin_primary_contact_number = '0817189287'
)[0]

social_profile = models.SocialProfiles.objects.get_or_create(
    name="Twitter",
    profile_page="https://twitter.com/LloydDoctor"
)[0]

abuser = models.Abuser.objects.get_or_create(
    full_names = "Doctor Lloyd Munyai",
    social_profiles = social_profile,
    contact_details = contact_numbers,
    workplace_details = "Director of Mobile at Datawizzards as of October 2020")[0]

case = models.Case.objects.get_or_create(
    title = "Groping Charge",
    report_type = "REPORT",
    occasion = "EVENT",
    preview = "Doctor groped me at a party on Feb 15, 2020. He invited me to a friend's party where he pretended that we were dating, thereby groping me in many ways without respectig my NO!",
    when = "2020-02-15",
    reporter = quser,
    frequency = 2,
    perpetrators = abuser,
    case_status = "VERIFIED",
    settlement_type = "OUT OF COURT",
    police_case_number = "2020/02/123")[0]
case

<Case: Case object (2)>

In [67]:
full_names = "nkosi"
full_names

'nkosi'

In [68]:
models.Case.objects.all()

<QuerySet [<Case: Case object (1)>]>

In [69]:
models.Case.objects.filter(perpetrators__full_names__icontains=full_names)

<QuerySet [<Case: Case object (1)>]>

# Load Cases from File

In [23]:
cases_df = pd.read_excel("~/Google Drive/dev/Safe@Home/Cases/Book1.xlsx")
cases_df

,Title,Type,Occasion,Preview,When?,Case Status,Settlement Type,Case Number,Perpatrator,URL
0,Rape Charge,REPORT,EVENT,The full background facts were comprehensively...,1998-09-20,VERIFIED,IN COURT,CCT 323/18,JABULANE ALPHEUS TSHABALALA,NaN
1,Rape Charge,REPORT,EVENT,The full background facts were comprehensively...,1998-09-21,VERIFIED,IN COURT,CCT 69/19,ANNANIUS NTULI,NaN


In [24]:
for case in cases_df.values:
    title = case[0]
    report_type = case[1]
    ocassion = case[2]
    preview = case[3]
    when = case[4]
    status = case[5]
    settlement = case[6]
    case_num = case[7]
    url = case[9]

    abuser = case[8].title()
    abuser = models.Abuser.objects.get_or_create(
        full_names = abuser)[0]
    
    cs = models.Case.objects.update_or_create(
        title = title,
        report_type = report_type,
        occasion = ocassion,
        preview = preview,
        when = when,
        reporter = quser,
        perpetrators = abuser,
        frequency = 1,
        case_status = status,
        settlement_type = settlement,
        police_case_number = case_num)[0]
    # cs.perpetrators.add(abuser)
    print(cs)

Case object (3)
Case object (4)


# FetchMe

In [6]:
address = addresses[np.random.randint(len(addresses))]
address

<Address: Address object (25)>

In [7]:
from datetime import datetime, timedelta

In [8]:
models.ContactDetails.objects.all().values()

<QuerySet [{'id': 1, 'primary_contact_number': '0817189287', 'secondary_contact_number': '0798595080', 'next_of_kin_firstname': 'Mabu', 'next_of_kin_lastname': 'Manaileng', 'next_of_kin_primary_contact_number': '0798595080', 'next_of_kin_secondary_contact_number': None}, {'id': 2, 'primary_contact_number': '0727308412', 'secondary_contact_number': None, 'next_of_kin_firstname': None, 'next_of_kin_lastname': None, 'next_of_kin_primary_contact_number': None, 'next_of_kin_secondary_contact_number': None}, {'id': 3, 'primary_contact_number': '0716634002', 'secondary_contact_number': None, 'next_of_kin_firstname': None, 'next_of_kin_lastname': None, 'next_of_kin_primary_contact_number': None, 'next_of_kin_secondary_contact_number': None}]>

In [9]:
models.FetchMeIncident.objects.all().delete()

(0, {})

In [10]:
inc = models.FetchMeIncident.objects.create(
    incident_type="NOW",
    pickup_address=address,
    pickup_time=datetime.now(),
    victim_full_names="Mabu Manaileng",
    contact_details=models.ContactDetails.objects.get(id=1)
)
inc

Singal Recieved
Victim: Mabu Manaileng 
Location: 626 Kamdebo Rd, Florauna, Pretoria, 0182, South Africa


<FetchMeIncident: FetchMeIncident object (9)>

In [8]:
inc.id

7

In [11]:
incident = models.FetchMeIncident.objects.get(id=inc.id)
incident

<FetchMeIncident: FetchMeIncident object (7)>

In [12]:
incident.victim_full_names

'Mabu Manaileng'

In [15]:
incident.pickup_address.lattitude

'-26.2472189'

In [30]:
models.FetchMeIncident.objects.all().values()

<QuerySet [{'id': 1, 'incident_type': 'NOW', 'incident_status': 'WAITING', 'pickup_address_id': 17, 'pickup_time': datetime.datetime(2020, 10, 18, 15, 4, 5, 294305, tzinfo=<UTC>), 'victim_full_names': 'Mabu Manaileng', 'contact_details_id': 1}]>

In [32]:
inc[0].save()